In [0]:
%sql
CREATE CATALOG IF NOT EXISTS my_catalog
COMMENT 'This is my Unity Catalog for managing data governance';

CREATE SCHEMA IF NOT EXISTS my_catalog.my_schema
COMMENT 'This is my schema for organizing tables';

CREATE VOLUME IF NOT EXISTS my_catalog.my_schema.my_volume
COMMENT 'This volume is used to store data files';


In [0]:
%sql
CREATE TABLE IF NOT EXISTS my_catalog.my_schema.my_table
USING PARQUET
LOCATION 'volume://my_catalog/my_schema/my_volume/';


org.apache.spark.sql.AnalysisException: [UC_FILE_SCHEME_FOR_TABLE_CREATION_NOT_SUPPORTED] Creating table in Unity Catalog with file scheme volume is not supported.
Instead, please create a federated data source connection using the CREATE CONNECTION command for the same table provider, then create a catalog based on the connection with a CREATE FOREIGN CATALOG command to reference the tables therein. SQLSTATE: 0AKUC
	at com.databricks.unity.CredentialScopeSQLHelper$.checkPathOperations(CredentialScopeSQLHelper.scala:126)
	at com.databricks.unity.CredentialScopeSQLHelper$.register(CredentialScopeSQLHelper.scala:178)
	at com.databricks.unity.CredentialScopeSQLHelper$.registerCreateTableAccess(CredentialScopeSQLHelper.scala:889)
	at com.databricks.sql.managedcatalog.CredentialScopeTableCredentialHandler.injectCredential(ResolveWithCredential.scala:514)
	at com.databricks.sql.managedcatalog.ResolveWithCredential.com$databricks$sql$managedcatalog$ResolveWithCredential$$maybeDecorateCatalogT

In [0]:
%sql
SELECT * FROM cloud_files('dbfs:/Volumes/my_catalog/my_schema/my_volume/myfile.parquet', 'PARQUET');


com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: java.lang.IllegalArgumentException: cloudFiles.schemaLocation Could not find required option: schemaLocation. Please provide a schema location using `cloudFiles.schemaLocation` for storing inferred schema and supporting schema evolution.
	at com.databricks.sql.cloudfiles.options.FileNotificationSourceOptionParser.getOption(CloudFilesOptionsBase.scala:1445)
	at com.databricks.sql.cloudfiles.options.FileNotificationSourceOptionParser.getOption$(CloudFilesOptionsBase.scala:1436)
	at com.databricks.sql.cloudfiles.options.CloudFilesAuthOptions.com$databricks$sql$cloudfiles$options$CloudFilesSourceOptionParser$$super$getOption(CloudFilesAuthOptions.scala:47)
	at com.databricks.sql.cloudfiles.options.CloudFilesSourceOptionParser.getOption(CloudFilesOptionsBase.scala:1420)
	at com.databricks.sql.cloudfiles.options.CloudFilesSourceOptionParser.getOption$(CloudFilesOptionsBase.scala:1414)
	at com.databricks.sql.cloudf

In [0]:
%sql
SHOW VOLUMES IN my_catalog.my_schema


database,volume_name
my_schema,my_volume


In [0]:
%sql
SELECT * FROM read_files(
  'dbfs:/Volumes/my_catalog/my_schema/my_volume/my_data.parquet',
  FORMAT => 'parquet'
);

com.databricks.sql.cloudfiles.errors.CloudFilesException: Cannot read files when the input path `dbfs:/Volumes/my_catalog/my_schema/my_volume/my_data.parquet` does not exist. Please make sure the input path exists and re-try.
	at com.databricks.sql.cloudfiles.errors.CloudFilesErrors$.pathDoesNotExistForReadFiles(CloudFilesErrors.scala:254)
	at com.databricks.sql.cloudfiles.utils.SchemaInferenceUtils$.$anonfun$sampleData$1(SchemaInferenceUtils.scala:204)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at com.databricks.sql.cloudfiles.utils.SchemaInferenceUtils$.sampleData(SchemaInferenceUtils.scala:170)
	at com.databricks.sql.cloudfiles.CloudFilesAnalysis.rewriteInternal(CloudFilesAnalysis.scala:228)
	at com.databricks.sql.cloudfiles.CloudFilesAnalysis.$anonfun$rewriteReadFiles$1(CloudFilesAnalysis.scala:188)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at com.databricks.sql.cloudfiles.CloudFilesAnalysis.com$databricks$sql$clo

In [0]:
%sql

SHOW FILES IN my_catalog.my_schema.my_volume;


org.apache.spark.sql.catalyst.parser.ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'IN': missing 'FUNCTIONS'. SQLSTATE: 42601 (line 1, pos 11)

== SQL ==
SHOW FILES IN my_catalog.my_schema.my_volume
-----------^^^

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(parsers.scala:301)
	at org.apache.spark.sql.catalyst.parser.AbstractParser.parse(parsers.scala:114)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:122)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(AbstractSqlParser.scala:87)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:77)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:98)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:74)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$5(SparkSession.scala:944)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.sc

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Read the Parquet file from the Unity Catalog Volume
df = spark.read.parquet("dbfs:/Volumes/my_catalog/my_schema/my_volume/my_data.parquet")

# Display the DataFrame
df.display()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-6200541398754765>, line 7
      4 spark = SparkSession.builder.getOrCreate()
      6 # Read the Parquet file from the Unity Catalog Volume
----> 7 df = spark.read.parquet("dbfs:/Volumes/my_catalog/my_schema/my_volume/my_data.parquet")
      9 # Display the DataFrame
     10 df.display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:634, in DataFrameReader.parquet(self, *paths, **options)
    623 int96RebaseMode = options.get("int96RebaseMode", No

In [0]:
df = spark.read.parquet("dbfs:/Volumes/my_catalog/my_schema/my_volume/myfile.parquet")
df.display()


job_id,job_type_id,received_time_stamp,source_reference,source_id,key_attributes,source_payload,job_status,s3_filename,job_step_status,correlation_id,message_metadata,created_on_dt,updated_on_dt,reprocessing_status,reprocess_from_step,job_process_seq,topic_message,op_val,hvr_last_upd_tms,trans_order_key
20740986,8,1720479615366,a0H8b00000q1B0LEAU,null,null,"{""Bill To Address1"":""12 RUE DU PORT"",""Bill To Address2"":null,""Bill To Address3"":null,""Bill To Address4"":""(POST)"",""Bill To City"":""NANTERRE"",""Bill To Country"":""FR"",""Bill To Customer Account Site ID"":""280792922"",""Bill To Customer Name"":""BNP PARIBAS LEASE GROUP"",""Bill To Customer Number"":""246396"",""Bill To Postal Code"":""92000"",""Bill To Province"":null,""Bill To Site UCM Id"":""U-00382419"",""Bill To Site Use Id"":""6700354"",""Bill To State"":null,""CRM Asset Number"":""a0H8b00000q1B0LEAU"",""CRM Name"":""SMAX"",""De-Install Date"":null,""De-Installation reason"":null,""IB Status"":""Installed"",""Install Date"":""19-DEC-2023"",""Installed At Address1"":""18 RUE DU GENERAL CATROUX"",""Installed At Address2"":null,""Installed At Address3"":null,""Installed At Address4"":null,""Installed At City"":""LIMOGES"",""Installed At Country"":""FR"",""Installed At Customer Name"":""CLINIQUE FRANCOIS CHENIEUX"",""Installed At Customer Number"":""439233"",""Installed At Postal Code"":""87000"",""Installed At Room ID"":null,""Installed At Site UCM Id"":""1-1FNKX3"",""Installed At Site Use Id"":""22775077"",""Installed At State"":null,""Installed at Customer Account Site ID"":""599702326"",""System ID"":""FRULS22258264"",""Expiry Date"":null,""Lot Number"":null,""Manufacturing Date"":""29-NOV-2023"",""PSI Code"":""UVS70W"",""PSI Description"":""VIVID S70 N R6 - NOR"",""Sales Order Number"":""5378284"",""Serial Number"":""125646S70N"",""Ship To Address1"":""18 RUE DU GENERAL CATROUX"",""Ship To Address2"":null,""Ship To Address3"":null,""Ship To Address4"":null,""Ship To City"":""LIMOGES"",""Ship To Country"":""FR"",""Ship To Customer Account Site ID"":""599702326"",""Ship To Customer Name"":""CLINIQUE FRANCOIS CHENIEUX"",""Ship To Customer Number"":""439233"",""Ship To Postal Code"":""87000"",""Ship To Province"":null,""Ship To Site UCM Id"":""1-1FNKX3"",""Ship To Site Use Id"":""22775077"",""Ship To State"":null,""UDI"":""(01)00195278505255(11)231100(21)125646S70N"",""IOI Instance Id"":""2012364787"",""Record Type"":""GEHCS"",""Parent IP Id"":null,""Parent IOI Instance Id"":null,""Related MDUs Details"":null}",3,inbound/CRM/ibrInboundSMAXIPDetailsBucket/a0H8b00000q1B0LEAU_20240708230009.json,531,null,"{""key"": ""a0H8b00000q1B0LEAU"", ""topic"": ""ibrInboundCrmIPDetailsTopic"", ""offsetId"": 1676394, ""partition"": 2}",2024-07-08T23:00:16.424Z,2024-07-08T23:00:16.694Z,0,null,1,"{ ""CRM Asset Number"" : ""a0H8b00000q1B0LEAU"", ""IOI Instance Id"" : ""2012364787"", ""Ship To Country"" : ""FR"", ""Installed At Country"" : ""FR"", ""CRM Name"" : ""SMAX"", ""Bucket Name"" : ""ibr-files"", ""File Name"" : ""inbound/CRM/ibrInboundSMAXIPDetailsBucket/a0H8b00000q1B0LEAU_20240708230009.json"", ""IB Status"" : ""Installed"", ""Modality Code"" : ""UVS70W"" }",2,2024-07-09T00:05:04.231Z,668C7EB000000001
20740981,8,1720479615257,a0H8b00000s7IvIEAU,null,null,"{""Bill To Address1"":""364 SYDNEY ROAD"",""Bill To Address2"":null,""Bill To Address3"":null,""Bill To Address4"":null,""Bill To City"":""COBURG"",""Bill To Country"":""AU"",""Bill To Customer Account Site ID"":""280207979"",""Bill To Customer Name"":""COBURG MEDICAL IMAGING PTY LTD ATF COBURG MEDICAL IMAGING TRUST"",""Bill To Customer Number"":""4295607"",""Bill To Postal Code"":""3058"",""Bill To Province"":null,""Bill To Site UCM Id"":""U-6SQQ3I"",""Bill To Site Use Id"":""6114312"",""Bill To State"":""VIC"",""CRM Asset Number"":""a0H8b00000s7IvIEAU"",""CRM Name"":""SMAX"",""De-Install Date"":null,""De-Installation reason"":null,""IB Status"":""Installed"",""Install Date"":""22-FEB-2024"",""Installed At Address1

In [0]:
df = spark.read.parquet("dbfs:/Volumes/my_catalog/my_schema/my_volume/myfile.parquet")
df.createOrReplaceTempView("my_temp_table")



In [0]:
%sql

select * from my_temp_table

job_id,job_type_id,received_time_stamp,source_reference,source_id,key_attributes,source_payload,job_status,s3_filename,job_step_status,correlation_id,message_metadata,created_on_dt,updated_on_dt,reprocessing_status,reprocess_from_step,job_process_seq,topic_message,op_val,hvr_last_upd_tms,trans_order_key
20740986,8,1720479615366,a0H8b00000q1B0LEAU,null,null,"{""Bill To Address1"":""12 RUE DU PORT"",""Bill To Address2"":null,""Bill To Address3"":null,""Bill To Address4"":""(POST)"",""Bill To City"":""NANTERRE"",""Bill To Country"":""FR"",""Bill To Customer Account Site ID"":""280792922"",""Bill To Customer Name"":""BNP PARIBAS LEASE GROUP"",""Bill To Customer Number"":""246396"",""Bill To Postal Code"":""92000"",""Bill To Province"":null,""Bill To Site UCM Id"":""U-00382419"",""Bill To Site Use Id"":""6700354"",""Bill To State"":null,""CRM Asset Number"":""a0H8b00000q1B0LEAU"",""CRM Name"":""SMAX"",""De-Install Date"":null,""De-Installation reason"":null,""IB Status"":""Installed"",""Install Date"":""19-DEC-2023"",""Installed At Address1"":""18 RUE DU GENERAL CATROUX"",""Installed At Address2"":null,""Installed At Address3"":null,""Installed At Address4"":null,""Installed At City"":""LIMOGES"",""Installed At Country"":""FR"",""Installed At Customer Name"":""CLINIQUE FRANCOIS CHENIEUX"",""Installed At Customer Number"":""439233"",""Installed At Postal Code"":""87000"",""Installed At Room ID"":null,""Installed At Site UCM Id"":""1-1FNKX3"",""Installed At Site Use Id"":""22775077"",""Installed At State"":null,""Installed at Customer Account Site ID"":""599702326"",""System ID"":""FRULS22258264"",""Expiry Date"":null,""Lot Number"":null,""Manufacturing Date"":""29-NOV-2023"",""PSI Code"":""UVS70W"",""PSI Description"":""VIVID S70 N R6 - NOR"",""Sales Order Number"":""5378284"",""Serial Number"":""125646S70N"",""Ship To Address1"":""18 RUE DU GENERAL CATROUX"",""Ship To Address2"":null,""Ship To Address3"":null,""Ship To Address4"":null,""Ship To City"":""LIMOGES"",""Ship To Country"":""FR"",""Ship To Customer Account Site ID"":""599702326"",""Ship To Customer Name"":""CLINIQUE FRANCOIS CHENIEUX"",""Ship To Customer Number"":""439233"",""Ship To Postal Code"":""87000"",""Ship To Province"":null,""Ship To Site UCM Id"":""1-1FNKX3"",""Ship To Site Use Id"":""22775077"",""Ship To State"":null,""UDI"":""(01)00195278505255(11)231100(21)125646S70N"",""IOI Instance Id"":""2012364787"",""Record Type"":""GEHCS"",""Parent IP Id"":null,""Parent IOI Instance Id"":null,""Related MDUs Details"":null}",3,inbound/CRM/ibrInboundSMAXIPDetailsBucket/a0H8b00000q1B0LEAU_20240708230009.json,531,null,"{""key"": ""a0H8b00000q1B0LEAU"", ""topic"": ""ibrInboundCrmIPDetailsTopic"", ""offsetId"": 1676394, ""partition"": 2}",2024-07-08T23:00:16.424Z,2024-07-08T23:00:16.694Z,0,null,1,"{ ""CRM Asset Number"" : ""a0H8b00000q1B0LEAU"", ""IOI Instance Id"" : ""2012364787"", ""Ship To Country"" : ""FR"", ""Installed At Country"" : ""FR"", ""CRM Name"" : ""SMAX"", ""Bucket Name"" : ""ibr-files"", ""File Name"" : ""inbound/CRM/ibrInboundSMAXIPDetailsBucket/a0H8b00000q1B0LEAU_20240708230009.json"", ""IB Status"" : ""Installed"", ""Modality Code"" : ""UVS70W"" }",2,2024-07-09T00:05:04.231Z,668C7EB000000001
20740981,8,1720479615257,a0H8b00000s7IvIEAU,null,null,"{""Bill To Address1"":""364 SYDNEY ROAD"",""Bill To Address2"":null,""Bill To Address3"":null,""Bill To Address4"":null,""Bill To City"":""COBURG"",""Bill To Country"":""AU"",""Bill To Customer Account Site ID"":""280207979"",""Bill To Customer Name"":""COBURG MEDICAL IMAGING PTY LTD ATF COBURG MEDICAL IMAGING TRUST"",""Bill To Customer Number"":""4295607"",""Bill To Postal Code"":""3058"",""Bill To Province"":null,""Bill To Site UCM Id"":""U-6SQQ3I"",""Bill To Site Use Id"":""6114312"",""Bill To State"":""VIC"",""CRM Asset Number"":""a0H8b00000s7IvIEAU"",""CRM Name"":""SMAX"",""De-Install Date"":null,""De-Installation reason"":null,""IB Status"":""Installed"",""Install Date"":""22-FEB-2024"",""Installed At Address1